In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
#fixed para
batch_size = 32
clip = 2
lr = 1e-4
reuse = False

In [2]:
# model parameters
block = schnet_block
head = cat3Head_type
data = '../Data/{}_data_ACSF_expand_PCA.pickle'
dim = 512
layer1 = 3
layer2 = 3
factor = 2
interleave = True
aggr = 'max'

In [3]:
types = [0,1,2,3,4]
epochs = [70] * 5
#epochs = [2] * 7

In [4]:
prefix = '_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                        for i in [reuse,block,head,data,dim,layer1,layer2,factor]])

In [5]:
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')

In [ ]:
# train model
for type_i,epoch in zip(types,epochs):
    print('\nstart type '+str(type_i))
    # load type data for all folds
    folds = []
    for f in range(5):
        with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(f)+'_type_'+str(type_i)+'.pickle', 'rb') as handle:
            folds.append(pickle.load(handle))
    folds = [[Data(**d) for d in fold] for fold in folds]
    
    # test data for type_i
    with open(data.format('test').split('pickle')[0][:-1]+'_type_'+str(type_i)+'.pickle', 'rb') as handle:
        test_data = pickle.load(handle)
    test_list = [Data(**d) for d in test_data]
    with open(data.format('test').split('pickle')[0][:-1]+'_id_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_id_test = pickle.load(handle)
    
    # fold
    for i in range(5):
        print('\nstart fold '+str(i))
        # parpare data
        train_list = []
        val_list = []
        for j in range(5):
            if i == j:
                val_list.extend(folds[j])
            else:
                train_list.extend(folds[j])
        train_dl = DataLoader(train_list,batch_size,shuffle=True)
        val_dl = DataLoader(val_list,batch_size,shuffle=False)        

        # train model
        model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]\
                    ,aggr=aggr,interleave=interleave).to('cuda:0')
        paras = trainable_parameter(model)
        opt = Adam(paras,lr=lr)
        scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)
        model,train_loss_perType,val_loss_perType = train_earlyStop(opt,model,epochs_type,train_dl,val_dl,paras,clip,True,\
                                                                    scheduler=scheduler)
        # predict oof
        with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(i)+'_type_'+str(type_i)+'_id.pickle', 'rb') as handle:
            test_id = pickle.load(handle)
            
        test_dl = DataLoader(val_list,batch_size,shuffle=False)
        model.eval()
        yhat_list = []
        with torch.no_grad():
            for data_torch in test_dl:
                data_torch = data_torch.to('cuda:0')
                yhat_list.append(model(data_torch,False,True))
        yhat = torch.cat(yhat_list).cpu().detach().numpy()        
        assert yhat.shape[0]==test_id.shape[0],'yhat and test_id should have same shape'
        
        # join
        submit_ = dict(zip(test_id,yhat))
        train_df['fold'+str(i)+'_type'+str(type_i)] = train_df.id.map(submit_)
        
        # predict test
        test_dl = DataLoader(test_list,batch_size,shuffle=False)
        model.eval()
        yhat_list = []
        with torch.no_grad():
            for data_torch in test_dl:
                data_torch = data_torch.to('cuda:0')
                yhat_list.append(model(data_torch,False,True))
        yhat = torch.cat(yhat_list).cpu().detach().numpy()  
        
        assert yhat.shape[0]==test_id_test.shape[0],'yhat and test_id should have same shape for test'
        # join
        submit_ = dict(zip(test_id_test,yhat))
        test_df['fold'+str(i)+'_type'+str(type_i)] = test_df.id.map(submit_)


start type 0
epoch:0, train_loss: +1.010, val_loss: +0.296, 
train_vector: +1.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.30|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.096, val_loss: +0.064, 
train_vector: +0.10|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.06|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

Training completed in 125.98496174812317s
epoch:0, train_loss: +1.034, val_loss: +0.164, 
train_vector: +1.03|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.16|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.094, val_loss: -0.047, 
train_vector: +0.09|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : -0.05|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

Training completed in 126.48502922058105s
epoch:0, train_loss: +1.025, val_loss: +0.208, 
train_vector: +1.03|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.21|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.101

In [7]:
test['yhat'] = np.nanmean(test.iloc[:,5:],1)
#test = test[['id','yhat']]
test_df.to_csv('../Data/test_oof_'+prefix,index=False)

train['yhat'] = np.nanmean(train.iloc[:,6:],1)
#train = train[['id','yhat']]
train_df.to_csv('../Data/train_oof_'+prefix,index=False)